In [1]:
import pandas as pd
import hopsworks

In [2]:
project = hopsworks.login(project = "ID2223_Ernest")

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5476


In [3]:
data = pd.read_csv("test_data.csv")

data.head()

,date,city,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarradiation,solarenergy,uvindex,conditions,city_id,lon,lat,area,density,no_people
0,2023-01-13,biała podlaska,0.261852,-0.164531,0.175060,0.511956,0.511956,0.154474,0.630787,0.541200,...,-0.720730,-0.649502,-0.308263,5,biala_podlaska,52.033333,23.116667,49.40,1152.06,56942
1,2023-01-20,łomża,-1.942392,-1.707910,-1.812561,-2.097169,-2.097169,-2.116367,-1.854176,0.407207,...,0.963668,1.147782,1.008861,6,lomza,53.176389,22.073056,32.67,1915.30,62573
2,2023-01-14,biała podlaska,0.643355,0.269544,0.359955,0.779559,0.779559,0.794148,0.786097,0.206217,...,-0.640867,-0.649502,-0.308263,3,biala_podlaska,52.033333,23.116667,49.40,1152.06,56942
3,2023-01-15,biała podlaska,0.049905,-0.019840,0.128836,0.244353,0.244353,0.730180,0.164856,-0.128767,...,-1.163611,-1.057976,-0.308263,2,biala_podlaska,52.033333,23.116667,49.40,1152.06,56942
4,2023-01-16,biała podlaska,-0.204431,-0.019840,-0.009835,0.010201,0.010201,-0.389248,0.216627,0.507701,...,1.871211,1.801341,1.008861,3,biala_podlaska,52.033333,23.116667,49.40,1152.06,56942


In [24]:
data.city.unique()

array(['biała podlaska', 'łomża', 'białystok', 'bielsko-biała',
       'bydgoszcz', 'gdańsk', 'gorzów wielkopolski', 'kalisz', 'katowice',
       'koszalin', 'kraków', 'lublin', 'poznań', 'radom', 'rzeszów',
       'suwałki', 'szczecin', 'szczecinek', 'warszawa', 'wrocław'],
      dtype=object)

In [4]:
from functions.functions import get_model

data_pred = data[['date', 'city']].copy()

X = data[['tempmax', 'tempmin', 'temp', 'feelslikemax',
       'feelslikemin', 'feelslike', 'dew', 'humidity', 'precip', 'precipprob',
       'precipcover', 'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir',
       'cloudcover', 'visibility', 'solarradiation', 'solarenergy', 'uvindex',
       'conditions', 'area', 'density', 'no_people']]


model_gb = get_model(project=project,
                  model_name="Gradient_Duster",
                  evaluation_metric="mae",
                  sort_metrics_by="max")

model_rfr = get_model(project=project,
                  model_name="AirParticle_Forest",
                  evaluation_metric="mae",
                  sort_metrics_by="max")

model_lasso = get_model(project=project,
                  model_name="PM10Lasso",
                  evaluation_metric="mae",
                  sort_metrics_by="max")

data_pred['Gradient_Duster'] = model_gb.predict(X)
data_pred['AirParticle_Forest'] = model_rfr.predict(X)
data_pred['PM10Lasso'] = model_lasso.predict(X)

data_pred.head()

Connected. Call `.close()` to terminate connection gracefully.


,date,city,Gradient_Duster,AirParticle_Forest,PM10Lasso
0,2023-01-13,biała podlaska,21.981403,23.39,20.091883
1,2023-01-20,łomża,37.553505,35.29,27.338022
2,2023-01-14,biała podlaska,15.367046,18.06,14.886075
3,2023-01-15,biała podlaska,12.173279,13.69,15.643485
4,2023-01-16,biała podlaska,15.135148,17.12,17.780505


In [29]:
import geopandas
df = pd.read_excel("data_poland/meta.xlsx")

geometry = geopandas.points_from_xy(df.lat, df.lon)
geo_df = geopandas.GeoDataFrame(
    df, geometry=geometry
)
geo_df['city_lower'] = geo_df['city'].apply(lambda c: c.lower())
geo_df.head()

,city_id,city,lon,lat,area,density,no_people,geometry,city_lower
0,biala_podlaska,Biała Podlaska,52.033333,23.116667,49.4,1152.06,56942,POINT (23.11667 52.03333),biała podlaska
1,bialystok,Białystok,53.135278,23.145556,102.13,2872.90,293413,POINT (23.14556 53.13528),białystok
2,bielsko_biala,Bielsko-Biała,49.822500,19.044444,124.51,1351.90,168319,POINT (19.04444 49.82250),bielsko-biała
3,bydgoszcz,Bydgoszcz,53.125000,18.011111,175.98,1941.70,337666,POINT (18.01111 53.12500),bydgoszcz
4,gdansk,Gdańsk,54.347500,18.645278,263.44,1845.00,486022,POINT (18.64528 54.34750),gdańsk


In [40]:

preds = data_pred[data_pred.date == '2023-01-15']

In [43]:
len(preds['city'])

20